In [ ]:
import mod_sql
import pandas as pd

In [220]:
db = mod_sql.Database('localhost','root','1234','ubion7',3306)

In [ ]:
sql = """ 
select
`Country` as '국가',
`Units Sold` as '판매개수',
`Unit Price` as '가격',
`Unit Cost` as '원가',
(`Units Sold` * `Unit Price`) as '총수익',
(`Units Sold` * `Unit Cost`) as '총비용',
(`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
from 
`Sales Records`
"""
db.func_3(sql)

In [ ]:
df = pd.read_csv('../230331/230403/230404/csv/Sales Records.csv')

In [ ]:
df = df[['Country', 'Units Sold', 'Unit Price', 'Unit Cost']]

In [ ]:
df.columns = ['국가', '판매개수','가격','원가']

In [ ]:
df['총수익'] = df['판매개수'] * df['가격']
df['총비용'] = df['판매개수'] * df['원가']
df['총이윤'] = df['총수익'] - df['총비용'] 

### sql에서 반올림

In [ ]:
sql = """ 
select
`Total Cost`,
round(`Total Cost`, 1)
from
`Sales Records`
"""

db.func_3(sql)

### sql 그룹화

1. Country, Item Type을 기준으로 그룹화
2. Unit Sold의 평균, 합계 생성
3. Country를 기준으로 오름차순 정렬

In [ ]:
sql = """ 
select
`Country`,
`Item Type`,
AVG(`Units Sold`) as AVG,
SUM(`Units Sold`) as SUM
from
`Sales Records`
group by
`Country`,
`Item Type`
order by
`Country`
ASC

"""

db.func_3(sql)

In [ ]:
df2 = df[['Country','Item Type','Units Sold']]

In [ ]:
df2.groupby(['Country', 'Item Type']).agg(['mean','sum'])

In [ ]:
df2.groupby(['Country', 'Item Type']).agg(['mean','sum'])['Units Sold']

### dept table에서 부서의 이름(dname)이 sales인 소속 사원의 정보 출력
1. dept 테이블에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력

In [ ]:
sql = """ 
select
`deptno`
from
`dept`
where
`dname` = 'SALES'
"""

db.func_3(sql)

In [ ]:
sql = """ 
select
*
from
`emp`
where
`deptno` = 30
"""

db.func_3(sql)

In [ ]:
sql = """ 
select
*
from
`emp`
where
`deptno`= (
    select
    `deptno`
    from
    `dept`
    where
    `dname` = 'research'
)
"""

db.func_3(sql)

In [ ]:
sql = """ 
select
`deptno`
from
`dept`
where
`dname` in ('sales', 'research')
"""
db.func_3(sql)

In [ ]:
sql = """ 
select
*
from
`emp`
where
`deptno` in (
    select
    `deptno`
    from
    `dept`
    where
    `dname` in ('sales', 'research')
)
"""
db.func_3(sql)

In [174]:
dep = pd.read_csv('../230331/230403/230404/csv/dept.csv')
dep

,DEPTNO,DNAME,LOC
0,10.0,ACCOUNTING,NEW YORK
1,20.0,RESEARCH,DALLAS
2,30.0,SALES,CHICAGO
3,40.0,OPERATIONS,BOSTON


In [241]:
emp = pd.read_csv('../230331/230403/230404/csv/emp.csv')

In [162]:
flag = dep.loc[(dep['DNAME'] == 'RESEARCH') | (dep['DNAME'] == 'SALES'),'DEPTNO'].tolist()
flag

[20.0, 30.0]

In [163]:
emp.loc[emp['DEPTNO'].isin(flag)].reset_index(drop = True)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L,L/H
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,LOW,LOW
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1,LOW,LOW
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1,LOW,LOW
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,HIGH,HIGH
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1,LOW,LOW
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2,HIGH,HIGH
6,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,HIGH,HIGH
7,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1,LOW,LOW
8,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,LOW,LOW
9,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,LOW,LOW


1. emp 데이터프레임에서 JOB 컬럼의 데이터를 확인하여 새로운 파생변수 생성
2. job_code SALESMAN이면 1, MANAGER면 2, 나머지는 3으로 하여 새로운 파생변수에 대입

In [ ]:
emp['job_code'] = 3
for i in emp.index:
    if emp.loc[i,'JOB'] == 'SALESMAN':
       emp.loc[i,'job_code'] = 1        
    elif emp.loc[i,'JOB'] == 'MANAGER':
       emp.loc[i,'job_code'] = 2
    


In [ ]:
emp['JOB'].apply(lambda x : 1 if (x == 'SALESMAN') else (2 if (x == 'MANAGER') else 3))

In [142]:
def change(x):
    if x == 'SALESMAN':
        result = 1
    elif x == 'MANAGER':
        result = 2
    else:
        result = 3
    return result

emp['job_code'] = emp['JOB'].apply(change)

In [145]:
sql = """ 
select 
`EMPNO`,
`ENAME`,
`JOB`,
CASE
    WHEN `JOB` = 'SALESMAN'
    THEN 1
    WHEN `JOB` = 'MANAGER' 
    THEN 2
    ELSE 3
END as `job_code`
from
emp
"""

db.func_3(sql)

,EMPNO,ENAME,JOB,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


In [147]:
sql = """ 
select
`EMPNO`,
`ENAME`,
`JOB`,
if(`JOB` = 'SALESMAN', 1, if(`JOB` = 'MANAGER',2,3)) as `JOB_CODE`
from
emp

"""

db.func_3(sql)

,EMPNO,ENAME,JOB,JOB_CODE
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


1. emp 데이터에서 파생변수 H/L 생성
2. SAL이 2000 이상이면 HIGH, 아니면 LOW로 데이터를 삽입

In [148]:
sql = """ 
select
`SAL`,
IF(`SAL` >= 2000, 'HIGH','LOW') as `H/L`
from
emp
"""
db.func_3(sql)

,SAL,H/L
0,800.0,LOW
1,1600.0,LOW
2,1250.0,LOW
3,2975.0,HIGH
4,1250.0,LOW
5,2850.0,HIGH
6,2450.0,HIGH
7,3000.0,HIGH
8,5000.0,HIGH
9,1500.0,LOW


In [150]:
for i in emp.index:
    if emp.loc[i,'SAL'] >= 2000:
        emp.loc[i,'H/L'] = 'HIGH'
    else:
        emp.loc[i,'H/L'] = 'LOW'

In [152]:
emp[['SAL','H/L']]

,SAL,H/L
0,800.0,LOW
1,1600.0,LOW
2,1250.0,LOW
3,2975.0,HIGH
4,1250.0,LOW
5,2850.0,HIGH
6,2450.0,HIGH
7,3000.0,HIGH
8,5000.0,HIGH
9,1500.0,LOW


In [153]:
emp['L/H'] = emp['SAL'].apply(lambda x : 'HIGH' if (x >= 2000) else 'LOW')

In [155]:
sql = """ 
select 
`EMPNO`,
`ENAME`,
`SAL`,
CASE
    WHEN `SAL` >= 2000
    THEN 'HIGH'
    ELSE 'LOW'
END as `L/H`
from
emp
"""

db.func_3(sql)

,EMPNO,ENAME,SAL,L/H
0,7369.0,SMITH,800.0,LOW
1,7499.0,ALLEN,1600.0,LOW
2,7521.0,WARD,1250.0,LOW
3,7566.0,JONES,2975.0,HIGH
4,7654.0,MARTIN,1250.0,LOW
5,7698.0,BLAKE,2850.0,HIGH
6,7782.0,CLARK,2450.0,HIGH
7,7788.0,SCOTT,3000.0,HIGH
8,7839.0,KING,5000.0,HIGH
9,7844.0,TURNER,1500.0,LOW


### join

1. emp와 dept table을 결합

In [176]:
sql = """ 
select
*
from
emp
right join 
dept
on
emp.DEPTNO = dept.DEPTNO
"""

db.func_3(sql)


,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
2,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
3,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0,20.0,RESEARCH,DALLAS
4,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,20.0,RESEARCH,DALLAS
5,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20.0,RESEARCH,DALLAS
6,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20.0,RESEARCH,DALLAS
7,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20.0,RESEARCH,DALLAS
8,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,30.0,SALES,CHICAGO
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO


In [217]:
sql = """ 
select

emp.EMPNO,
emp.ENAME,
emp.DEPTNO,
dept.DEPTNO,
dept.DNAME,
dept.LOC

from
emp
inner join 
dept
on
emp.DEPTNO = dept.DEPTNO
where
loc not in ('NEW YORK','DALLAS')


"""

db.func_3(sql)


,EMPNO,ENAME,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7499.0,ALLEN,30.0,30.0,SALES,CHICAGO
1,7521.0,WARD,30.0,30.0,SALES,CHICAGO
2,7654.0,MARTIN,30.0,30.0,SALES,CHICAGO
3,7698.0,BLAKE,30.0,30.0,SALES,CHICAGO
4,7844.0,TURNER,30.0,30.0,SALES,CHICAGO
5,7900.0,JAMES,30.0,30.0,SALES,CHICAGO


In [175]:
pd.merge(emp,dep, on = 'DEPTNO', how = 'left')

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,SALES,CHICAGO
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,SALES,CHICAGO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,SALES,CHICAGO
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,SALES,CHICAGO
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,RESEARCH,DALLAS
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,SALES,CHICAGO


### 문제
- 부서의 위치가 'NEW YORK', 'DALLAS'가 아닌 소속 사원의 정보를 출력

In [203]:
sql = """ 
select
DEPTNO
from
dept
where
`LOC` not in ('NEW YORK', 'DALLAS')
"""

db.func_3(sql)

,DEPTNO
0,30.0
1,40.0


In [216]:
sql = """ 
select
*
from
emp
where
`DEPTNO` in(select
`DEPTNO`
from
dept
where
`LOC` not in ('NEW YORK', 'DALLAS')
    )
"""
db.func_3(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


### 테이블의 결합

1. 유니언 결합 (단순 행결합)
2. 조인 결합

In [222]:
sql = """ 
select * from tran_1
union
select * from tran_2
"""

db.func_3(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [223]:
sql = """ 
select * from tran_d_1
union
select * from tran_d_2
"""

db.func_3(sql)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [224]:
sql = """ 
select * from tran_1 left join tran_d_1 on tran_1.transaction_id = tran_d_1.transaction_id
"""
db.func_3(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d_1.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0.0,T0000000113,S005,1.0
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1.0,T0000000114,S001,1.0
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2.0,T0000000115,S003,1.0
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3.0,T0000000116,S005,1.0
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4.0,T0000000117,S002,2.0
...,...,...,...,...,...,...,...,...
5238,T0000005108,210000,2019-06-15 02:42:41,HD315748,NaN,None,None,NaN
5239,T0000005109,150000,2019-06-15 03:36:16,HI215420,NaN,None,None,NaN
5240,T0000005110,50000,2019-06-15 03:44:06,IK880102,NaN,None,None,NaN
5241,T0000005111,210000,2019-06-15 04:14:06,IK074758,NaN,None,None,NaN


In [225]:
sql = """ 
select * from
(select * from tran_1
union
select * from tran_2) as tran
left join
(select * from tran_d_1
union
select * from tran_d_2) as tran_d
on tran.transaction_id = tran_d.transaction_id
"""

db.func_3(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006887,210000,2019-07-31 19:43:42,AS961301,7132,T0000006887,S005,1
7140,T0000006888,210000,2019-07-31 20:05:45,GD025393,7133,T0000006888,S005,1
7141,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7142,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1


In [227]:
tran1 = pd.read_csv('../230331/230403/230404/csv/tran_1.csv')
tran2 = pd.read_csv('../230331/230403/230404/csv/tran_2.csv')
trand1 = pd.read_csv('../230331/230403/230404/csv/tran_d_1.csv')
trand2 = pd.read_csv('../230331/230403/230404/csv/tran_d_2.csv')

In [233]:
tran_a = pd.concat([tran1,tran2])
tran_a

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
1781,T0000006894,180000,2019-07-31 21:20:44,HI400734
1782,T0000006895,85000,2019-07-31 21:52:48,AS339451
1783,T0000006896,100000,2019-07-31 23:35:25,OA027325
1784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [231]:
tran_b = pd.concat([trand1,trand2])

In [236]:
pd.merge(tran_a,tran_b, on = ['transaction_id'], how = 'left')

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,S002,2
...,...,...,...,...,...,...,...
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1


### 문자열이 포함되어 있는 경우 출력


In [240]:
sql = """ 
select
*
from
emp
where
ENAME like '%%S%%'
"""

db.func_3(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
2,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
4,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


1. pandas 이용하여 emp.csv에서 ENAME이 S로 시작하는 데이터 출력
2. S로 끝나는 데이터를 출력
3. S가 포함되어 있는 데이터를 출력

In [263]:
emp.loc[emp['ENAME'].str.endswith('S') == True] 

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [264]:
emp.loc[emp['ENAME'].str.startswith('S') == True] 

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0


In [266]:
emp.loc[emp['ENAME'].str.contains('S') == True]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0
